# EDA de **Telefonía Fija: Accesos por Provincia**

Cuaderno para **Visual Studio/VS Code** (Python) con pasos documentados en Markdown y exportación del CSV 'limpio'.

**Fuente:** CSV con columnas `Registro_ID, Provincia_ID, Año, Trimestre, Provincias, Hogares, Comercial, Gobierno, Otros, Total, TiempoTrim_ID`.

## 1. Importaciones y configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (11, 5),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
> **Ruta proporcionada:**

`C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Telefonia Fija\\Telefonía Fija Accesos Provincias_con_ID.csv`

Usa `encoding='utf-8-sig'` para preservar acentos/eñes en Windows.

In [ ]:
ruta = r'C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Telefonia Fija\\Telefonía Fija Accesos Provincias_con_ID.csv'

df = pd.read_csv(ruta, encoding='utf-8-sig')
df_raw = df.copy()

print('Shape:', df.shape)
print('Columnas:', list(df.columns))
display(df.head())

## 3. Perfil inicial
- Tipos de datos (`df.info()`)
- Nulos y duplicados
- Rangos básicos por métricas principales

In [ ]:
df.info()

print('
Nulos por columna:
', df.isna().sum())
print('Duplicados (filas completas):', df.duplicated().sum())

cols_rango = ['Hogares','Comercial','Gobierno','Otros','Total']
for col in cols_rango:
    if col in df.columns:
        print(col, 'min=', df[col].min(), 'max=', df[col].max(), 'mean=', float(df[col].mean()))

## 4. Validaciones de consistencia
- Conversión de columnas numéricas
- Chequeo de valores negativos
- Verificación **Total vs suma de componentes** y relleno de `Total` si falta
- Construcción de `Periodo` (YYYY-Qn)

In [ ]:
num_cols = ['Registro_ID','Provincia_ID','Año','Trimestre','Hogares','Comercial','Gobierno','Otros','Total','TiempoTrim_ID']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('
Negativos por columna:
', (df[num_cols] < 0).sum())

# Suma de componentes
if set(['Hogares','Comercial','Gobierno','Otros']).issubset(df.columns):
    comp_sum = df[['Hogares','Comercial','Gobierno','Otros']].sum(axis=1)
    # Rellenar Total si está faltante
    df['Total'] = df['Total'].fillna(comp_sum) if 'Total' in df.columns else comp_sum
    consist = (df['Total'] == comp_sum)
    print('Consistentes Total == suma:', consist.mean(), '(', int(consist.sum()), 'de', len(consist), ')')
    df['dif_total'] = df['Total'] - comp_sum

# Periodo
df['Periodo'] = pd.PeriodIndex.from_fields(year=df['Año'], quarter=df['Trimestre']).astype(str)

## 5. Limpieza y ordenamiento
- Eliminar duplicados
- Orden por `Año`, `Trimestre`, `Provincia_ID`
- Columnas auxiliares conservadas para auditoría (`dif_total`)

In [ ]:
df = df.drop_duplicates().sort_values(['Año','Trimestre','Provincia_ID']).reset_index(drop=True)
display(df.head())

## 6. Variables derivadas
- Porcentajes de composición sobre `Total`
- Top provincias por `Total` en el último trimestre


In [ ]:
total_nonzero = df['Total'].replace(0, pd.NA)
df['porc_hogares'] = (df['Hogares'] / total_nonzero) * 100
df['porc_comercial'] = (df['Comercial'] / total_nonzero) * 100
df['porc_gobierno'] = (df['Gobierno'] / total_nonzero) * 100
df['porc_otros'] = (df['Otros'] / total_nonzero) * 100

# Último trimestre disponible
ultimo = df[['Año','Trimestre']].max().to_dict()
filtro_ult = (df['Año'] == ultimo['Año']) & (df['Trimestre'] == ultimo['Trimestre'])
top_ult = df.loc[filtro_ult, ['Provincias','Total']].sort_values('Total', ascending=False).head(10)
print('Top 10 provincias por Total (último trimestre):')
display(top_ult)

## 7. Visualizaciones
- **Top 10** de provincias por `Total` (último trimestre)
- Serie temporal de `Total` para provincias seleccionadas
- Composición (%) por segmento para una provincia


In [ ]:
# 7.1 Top 10 provincias por Total en el último trimestre
plt.figure()
sns.barplot(data=top_ult, x='Provincias', y='Total', color='#2E86C1')
plt.xticks(rotation=45, ha='right')
plt.title('Top 10 provincias por Total (último trimestre)')
plt.xlabel('Provincia'); plt.ylabel('Total accesos'); plt.tight_layout(); plt.show()

# 7.2 Serie temporal para provincias clave
sel = ['Buenos Aires','CABA','Córdoba','Santa Fe']
df_sel = df[df['Provincias'].isin(sel)]
plt.figure()
sns.lineplot(data=df_sel, x='Periodo', y='Total', hue='Provincias', marker='o')
plt.xticks(rotation=45, ha='right')
plt.title('Serie temporal de Total por provincia seleccionada')
plt.xlabel('Periodo'); plt.ylabel('Total'); plt.legend(title='Provincia'); plt.tight_layout(); plt.show()

# 7.3 Composición porcentual para una provincia (ejemplo: Buenos Aires, último trimestre)
prov = 'Buenos Aires'
row = df.loc[filtro_ult & (df['Provincias'] == prov)].head(1)
if not row.empty:
    comp = row[['porc_hogares','porc_comercial','porc_gobierno','porc_otros']].T
    comp.columns = ['%']
    comp['segmento'] = comp.index.str.replace('porc_','')
    plt.figure()
    sns.barplot(data=comp, x='segmento', y='%')
    plt.gca().yaxis.set_major_formatter(lambda x, pos: f'{x:.0f}%')
    plt.title(f'Composición (%) de segmentos en {prov} (último trimestre)')
    plt.xlabel('Segmento'); plt.ylabel('% del total'); plt.tight_layout(); plt.show()

## 8. Guardado del archivo **limpio**
Se guarda el CSV en la **misma ruta** con la **leyenda `_limpio`** agregada al nombre, codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)